In [2]:
from numpy.random import randint
import matplotlib.pyplot as plt
from PIL import Image

class Stack:
    def __init__(self):
        self.lst = [] 
    
    def empty(self):
        return self.lst == [] 
    
    def push(self, x):
        self.lst.append(x)

    def pop(self):
        if self.empty():
            raise ValueError("empty stack") 
        return self.lst.pop()
    
def explore(maze, entrance, exit): 
    stack = Stack()
    stack.push(entrance)
    maze.tab[entrance[0]][entrance[1]].state = False 
    while True:
        i, j = stack.pop()
        if (i, j) == exit:
            break
        if j > 0 and maze.tab[i][j].S and maze.tab[i][j-1].state:
            stack.push((i, j)) 
            stack.push((i, j-1)) 
            maze.tab[i][j-1].state = False
        elif i < maze.p-1 and maze.tab[i][j].E and maze.tab[i+1][j].state: 
            stack.push((i, j))
            stack.push((i+1, j))
            maze.tab[i+1][j].state = False
        elif j < maze.q-1 and maze.tab[i][j].N and maze.tab[i][j+1].state: 
            stack.push((i, j))
            stack.push((i, j+1))
            maze.tab[i][j+1].state = False
        elif i > 0 and maze.tab[i][j].W and maze.tab[i-1][j].state: 
            stack.push((i, j))
            stack.push((i-1, j))
            maze.tab[i-1][j].state = False
    return stack.lst
    
class Cell:
    def __init__(self):
        self.N = False 
        self.W = False 
        self.S = False 
        self.E = False 
        self.state = False

class Maze:
    def __init__(self, p, q):
        self.p = p
        self.q = q
        self.tab = [[Cell() for j in range(q)] for i in range(p)]
    
    def show(self, entrance, exit, filename="maze.png"):
        for i in range(self.p-1):
            for j in range(self.q):
                if not self.tab[i][j].E:
                    plt.plot([i+1, i+1], [j, j+1], 'k', linewidth=2) 
        for j in range(self.q-1):
            for i in range(self.p):
                if not self.tab[i][j].N:
                    plt.plot([i, i+1], [j+1, j+1], 'k', linewidth=2) 
        
        i = 0
        for j in range(self.q):
            if j == entrance[1] and i == entrance[0]:
                continue
            if j == exit[1] and i == exit[0]:
                continue
            if not self.tab[i][j].W:
                plt.plot([i, i], [j, j+1], 'k', linewidth=2)
        j = 0
        for i in range(self.p):
            if i == entrance[0] and j == entrance[1]:
                continue
            if i == exit[0] and j == exit[1]:
                continue
            if not self.tab[i][j].S:
                plt.plot([i, i+1], [j, j], 'k', linewidth=2)
        
        i = self.p-1
        for j in range(self.q):
            if j == entrance[1] and i == entrance[0]:
                continue
            if j == exit[1] and i == exit[0]:
                continue
            if not self.tab[i][j].E:
                plt.plot([i+1, i+1], [j, j+1], 'k', linewidth=2)
        
        j = self.q-1
        for i in range(self.p):
            if i == entrance[0] and j == entrance[1]:
                continue
            if i == exit[0] and j == exit[1]:
                continue
            if not self.tab[i][j].N:
                plt.plot([i, i+1], [j+1, j+1], 'k', linewidth=2)

        plt.axis('equal')
        plt.xlim(0, self.p)
        plt.ylim(0, self.q)
        plt.axis('off')
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

        plt.savefig(filename, format="png", dpi=400)
        plt.close()
        
    def solution(self, entrance, exit, filename="maze.svg"):
        sol = explore(self, entrance, exit) 
        X, Y = [], []
        for (i, j) in sol:
            X.append(i+.5)
            Y.append(j+.5) 
        X.append(exit[0] + .5)
        Y.append(exit[1] + .5)
        plt.plot(X, Y, 'r', linewidth=2) 
        self.show(entrance, exit, filename)
        
def create(p, q, entrance, exit):
    maze = Maze(p, q)
    stack = Stack()
    i, j = entrance
    stack.push((i, j)) 
    maze.tab[i][j].state = True 
    while not stack.empty():
        i, j = stack.pop()
        v = []
        if j < q-1 and not maze.tab[i][j+1].state:
            v.append('N')
        if i > 0 and not maze.tab[i-1][j].state:
            v.append('W')
        if j > 0 and not maze.tab[i][j-1].state:
            v.append('S')
        if i < p-1 and not maze.tab[i+1][j].state:
            v.append('E') 
        if len(v) > 1:
            stack.push((i, j)) 
        if len(v) > 0:
            c = v[randint(len(v))] 
            if c == 'N':
                maze.tab[i][j].N = True
                maze.tab[i][j+1].S = True
                maze.tab[i][j+1].state = True 
                stack.push((i, j+1))
            elif c == 'W':
                maze.tab[i][j].W = True 
                maze.tab[i-1][j].E = True 
                maze.tab[i-1][j].state = True 
                stack.push((i-1, j))
            elif c == 'S':
                maze.tab[i][j].S = True 
                maze.tab[i][j-1].N = True 
                maze.tab[i][j-1].state = True 
                stack.push((i, j-1))
            else:
                maze.tab[i][j].E = True 
                maze.tab[i+1][j].W = True 
                maze.tab[i+1][j].state = True 
                stack.push((i+1, j))
    return maze

W, H = 25, 25
entrance = (2, 0)
exit = (W-1, H-1)
maze = create(W, H, entrance, exit)
maze.solution(entrance, exit)

n_patch = 5
left_to_right = True
last_exit = (0, 0)
for i in range(n_patch):
    j_range = range(n_patch) if left_to_right else list(reversed(range(n_patch)))
    for j in j_range:
        entrance = list(last_exit)
        
        if j == 0 and left_to_right:
            entrance[1] = 0
        elif j != 0 and left_to_right:
            entrance[0] = 0
        elif j == n_patch - 1 and not left_to_right:
            entrance[1] = 0
        elif j != n_patch - 1 and not left_to_right:
            entrance[0] = W-1

        if left_to_right:
            exit = (W-1, randint(1, H))
            if j == n_patch - 1:
                exit = (randint(1, W), H-1)
        else:
            exit = (0, randint(1, H))
            if j == 0:
                exit = (randint(1, W), H-1)

        if i == n_patch - 1 and j == n_patch - 1:
            exit = (W-1, H-1)

        last_exit = exit
    
        maze = create(W, H, entrance, exit)
        maze.solution(entrance, exit, filename=f"patch_{i}_{j}_sol.png")
        maze.show(entrance, exit, filename=f"patch_{i}_{j}.png")

    left_to_right = not left_to_right

patches = []
n_patch = 5
images = [Image.open(f"patch_{i}_{j}.png") for i in range(n_patch) for j in range(n_patch)]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
total_height = sum(heights)

new_im = Image.new('RGB', (heights[0] * n_patch, heights[0] * n_patch))

x_offset = 0
y_offset = 0

for i in range(n_patch):
    for j in range(n_patch):
        im = images[i * n_patch + j]
        new_im.paste(im, (x_offset, y_offset))
        x_offset += im.width
    y_offset += im.height
    x_offset = 0

new_im.save("all_patchs.png")

